In [1]:
import pandas as pd
import numpy as np
import pickle
import csv
import time
import os
from numba import njit
os.chdir('..')
from qmc import QMC
from mmnl import MMNL
os.chdir('./data')
import matplotlib.pyplot as plt

Load data in the format used by k. Train.

In [2]:
def load_data(path):
    dat = np.load(path)
    X = dat[:, :-1]
    Y = np.reshape(dat[:, -1], (-1, 1))
    return X, Y
X, Y = load_data('data.npy')

In [3]:
print(X)

[[  1.          0.          0.        ...   3.7         5.2000001
    3.4000002]
 [  1.          0.          0.        ...   4.3000001   5.2000001
    4.4      ]
 [  1.          0.          0.        ...   2.5         4.6
    4.8      ]
 ...
 [300.          0.          0.        ...   3.5999998   5.0000001
    2.5      ]
 [300.          0.          0.        ...   3.7         5.0000001
    2.8000001]
 [300.          0.          0.        ...   3.7         3.5
    3.4000002]]


In [4]:
def utilities(X, beta):
    #performs matrix product to obtain the probability of every row
    #X should be in format [display, feature, price]
    try:
        assert(X.shape == (11192, 5) and beta.shape == (3,300))
    except AssertionError:
        raise AssertionError('Ga X ff in juiste format gooien. X: %s, beta: %s' %(X.shape,beta.shape))
    beta_choice = np.zeros((3,11192))
    for i in range(11192):
        id = int(X[i,0])
        beta_choice[:,i] = beta[:,id-1]
    eps = np.random.gumbel(size=(11192,))
    P = (X[:,2:]@ beta_choice)[:,0] + eps
#     try:
#         p = P[0:4]
#         check = p/np.sum(p)
#         assert(np.sum(check) == 1. or np.sum(check) == 1)
#     except AssertionError:
#         raise AssertionError('Kansen van eerste aankoop sommeren niet naar 1 %f'%(np.sum(check)))

    Y = []
    for i in range(0,len(P), 4):
        choice = np.argmax(P[i:i+4])
#         p = P[i:i+4]
#         check = p/np.sum(p)
#         print(np.sum(check))
        Y.append(int(choice))
    return np.array(Y)

In [5]:
# @njit
def probs(X,beta):
    #performs matrix product to obtain the probability of every row
    #X should be in format [display, feature, price]
    if not (X.shape == (11192, 5) and beta.shape == (3,300)):
        raise AssertionError('Ga X ff in juiste format gooien.')
    beta_choice = np.zeros((3,11192))
    for i in range(11192):
        id = int(X[i,0])
        beta_choice[:,i] = beta[:,id-1]   
    P = np.exp((X[:,2:]@ beta_choice)[:,0])
        
    if not (P.shape == (11192,)):
        raise AssertionError('Product van X en beta gaat niet goed.')
#     try:
#         p = P[0:4]
#         check = p/np.sum(p)
#         assert(np.sum(check) == 1. or np.sum(check) == 1)
#     except AssertionError:
#         raise AssertionError('Kansen van eerste aankoop sommeren niet naar 1 %f'%(np.sum(check)))

    Y = np.zeros((2798,))
    for i in range(0,11192, 4):
        sum = P[i:i+4].sum(axis=0)
        Y[i//4] = (P[i:i+4]/sum).argmax(axis=0)
#         p = P[i:i+4]
#         check = p/np.sum(p)
#         print(np.sum(check))
    return Y

In [15]:
%%time
def dgp(X: np.ndarray, D, method):
    #X: dataset
    #D: amount of datasets
    np.random.seed(123)
    theta = np.array([1.5,  1.,  -1.1,  0.4,  0.1,  0.6])
    Y_array = np.zeros((2798,D))
    P = np.zeros((11192,D))
    if method == 'QMC':
        delta = QMC(300,3,D)
    elif method == 'SMC':
        delta = np.random.standard_normal((300,3,D))
    print(method)
    beta = theta[:3].reshape(-1,1) + delta * theta[3:].reshape(-1,1)
    for d in range(D):
        Y = []
        for row in X:
            uts = []
            for k in range(4):
                x = np.array([row[1 + k], row[5 + k], row[9 + k]])
                uts.append(x@ beta[int(row[0])-1,:,d] + np.random.gumbel())

            pred = np.argmax(uts)
            Y.append(pred)
        Y_array[:,d] = Y

    return Y_array
    

Wall time: 0 ns


In [16]:
%%time
D = 200
Y_dgp= dgp(X,D,'SMC')

SMC
Wall time: 16.6 s


In [17]:
pickle.dump( Y_dgp, open( "%i_%s_dgp.p"%(D,'SMC'), "wb" ) )